In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [13]:
df =  pd.read_csv('Data/DrugData.csv')

In [14]:
df_t = df.drop(['Unnamed: 0', 'urlDrugName','sentiment'], axis=1)

In [15]:
df_t.head(2)

,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,1,Highly Effective,Severe Side Effects,birth prevention,although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."


In [16]:
rating_rating= [[1,2,3,4], [5,6],[7,8,9,10]]

for i, idx in enumerate(rating_rating):
    df['rating'].replace(to_replace=idx, value = i-1, inplace=True)

    
rating_effectiveness = [['Ineffective'], ['Marginally Effective','Moderately Effective'],
                        ['Considerably Effective','Highly Effective']]

for i, idx in enumerate(rating_effectiveness):
    df['effectiveness'].replace(to_replace=idx, value= i, inplace=True)

rating_sideEffects = [['No Side Effects'], ['Mild Side Effects', 'Moderate Side Effects'],
                      ['Severe Side Effects','Extremely Severe Side Effects']]

for i, idx in enumerate(rating_sideEffects):
    df['sideEffects'].replace(to_replace=idx, value= i, inplace=True)

In [17]:
df.head(2)

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,sentiment
0,2202,enalapril,-1,2,1,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",0
1,3117,ortho-tri-cyclen,-1,2,2,birth prevention,although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",0


In [18]:

rating_effectiveness = [['Ineffective'], ['Marginally Effective','Moderately Effective'], ['Considerably Effective','Highly Effective']]
list1 = ['Ineffective', 'Moderate Effect', 'Highly Effective']



df_t['effectiveness'].replace(to_replace=['Ineffective'], value= 'Ineffective', inplace=True)

df_t['effectiveness'].replace(to_replace=['Marginally Effective','Moderately Effective'], value= 'Moderate Effect', inplace=True)

df_t['effectiveness'].replace(to_replace=['Considerably Effective','Highly Effective'], value= 'Highly Effective', inplace=True)

rating_sideEffects = [['No Side Effects'], ['Mild Side Effects', 'Moderate Side Effects'],['Severe Side Effects','Extremely Severe Side Effects']]
list2 = ['No Sideeffects', 'Moderate Sideeffects','High SideEffects' ]

df_t['sideEffects'].replace(to_replace='No Side Effects', value= 'No Sideeffects', inplace=True)

df_t['sideEffects'].replace(to_replace=['Mild Side Effects', 'Moderate Side Effects'], value= 'Moderate Sideeffects', inplace=True)

df_t['sideEffects'].replace(to_replace=['Severe Side Effects','Extremely Severe Side Effects'], value= 'High SideEffects' ,inplace=True)

In [19]:
df_t.head(3)

,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,4,Highly Effective,Moderate Sideeffects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,1,Highly Effective,High SideEffects,birth prevention,although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,10,Highly Effective,No Sideeffects,menstrual cramps,i was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...


## Distribution of sideEffects

In [21]:
frequency_side = df_t.groupby('sideEffects')[['sideEffects']].count()
frequency_side['%']= round(frequency_side['sideEffects']/frequency_side['sideEffects'].sum()*100,1)
frequency_side

,sideEffects,%
sideEffects,,
High SideEffects,746,18.0
Moderate Sideeffects,2199,53.1
No Sideeffects,1198,28.9


## Distribution of effectiveness

In [23]:
frequency_benefit = df_t.groupby('effectiveness')[['effectiveness']].count()
frequency_benefit['%']= round(frequency_benefit['effectiveness']/frequency_benefit['effectiveness'].sum()*100,1)
frequency_benefit

,effectiveness,%
effectiveness,,
Highly Effective,2979,71.9
Ineffective,329,7.9
Moderate Effect,835,20.2


## Distribution of Rating

In [26]:
dist_rating = df_t.groupby('rating')[['effectiveness']].count()
dist_rating.reset_index(inplace= True)

dist_rating['%']= round(dist_rating['effectiveness']/dist_rating['effectiveness'].sum()*100,1)
dist_rating

,rating,effectiveness,%
0,1,420,10.1
1,2,136,3.3
2,3,195,4.7
3,4,151,3.6
4,5,226,5.5
5,6,215,5.2
6,7,471,11.4
7,8,748,18.1
8,9,613,14.8
9,10,968,23.4


## Feature engineering and combing all the features

In [1]:
df_t['review']=(df_t['benefitsReview']+df_t['sideEffectsReview']+df_t['commentsReview'])
#df_t['Length of Review']= df_t.apply(lambda row: len(row['review']), axis=1)


NameError: name 'df_t' is not defined